# Pression à basse densité: Deuxième et troisième coefficients du Viriel

In [ ]:
import numpy,pandas,time,threading,os
from matplotlib import pyplot
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

La pression doit s'exprimer en fonction de la densité comme suit

$$ \frac {P}{nT} = 1 + B_2 n + B_3 n^2 + \mathcal{O}(n^3) $$

Les deuxième et troisième coefficients du viriel sont donnés par

$$ B_2 = \frac{16\pi R^3}{3} = \frac {16\pi}{3} = 16.75516 $$
$$ B_3 = \frac{160\pi^2 R^6}{9} = \frac{160\pi}{9} = 175.460 $$

Les termes correspondant valent

$$ B_2 n = 16.75516 \times 0.0002 = 3.351032 \times 10^{-3} $$
$$ B_3 n^2 = 175.460 \times 0.0002^2 = 7.018 \times 10^{-6} $$

Nous négligerons la contribution des coefficients d'ordre supérieur. 

L'équation d'état que nous devrions trouver numériquement pour la pression est donc 

$$ \frac {P}{nT} = 1.00335805 $$

## De longues simulations ou plusieurs petites ?

On teste ici quel est la stratégie la plus avantageuse pour minimiser l'erreur statistique parmi les suivantes: réaliser quelques longues simulations ou beaucoup de simulations plus courtes. Intuitivement, cela ne devrait rien changer car la pression pour une simulation est déjà calculée avec une moyenne (somme des $r_{ij}\cdot v_{ij}$ divisée par le temps de simulation).

In [ ]:
def petites_sim():
    
    NSim = 100 # nombre de simulations

    start = time.time()
    for i in range(NSim):
        print ("./simulationO3 3 3 3 0.0002 1 200 0 _Short    simulation n°", i)
        os.system("./simulationO3 3 3 3 0.0002 1 200 0 _Short")
        os.system("mkdir data_viriel")
        os.system("mv data/collisionData_Short.csv data_viriel/collisionData_LD_short{:d}.csv".format(i))
        os.system("mv data/infoSimulation_Short.csv data_viriel/infoSimulation_LD_short{:d}.csv".format(i))
        os.system("mv data/particle0Data_Short.csv data_viriel/particle0Data_LD_short{:d}.csv".format(i))
        os.system("mv data/excursionData_Short.csv data_viriel/excursionData_LD_short{:d}.csv".format(i))
    print ("Execution time: {:.3f} seconds".format(time.time()-start))

In [ ]:
def grandes_sim():
    
    NSim = 10 # nombre de simulations

    start = time.time()
    for i in range(NSim):
        print ("./simulationO3 3 3 3 0.0002 1 2000 0 _Long    simulation n°", i)
        os.system("./simulationO3 3 3 3 0.0002 1 2000 0 _Long")
        os.system("mkdir data_viriel")
        os.system("mv data/collisionData_Long.csv data_viriel/collisionData_LD_long{:d}.csv".format(i))
        os.system("mv data/infoSimulation_Long.csv data_viriel/infoSimulation_LD_long{:d}.csv".format(i))
        os.system("mv data/particle0Data_Long.csv data_viriel/particle0Data_LD_long{:d}.csv".format(i))
        os.system("mv data/excursionData_Long.csv data_viriel/excursionData_LD_long{:d}.csv".format(i))
    print ("Execution time: {:.3f} seconds".format(time.time()-start))

In [ ]:
threading.Thread(target=petites_sim).start()
threading.Thread(target=grandes_sim).start()

### Petites simulations

In [ ]:
N = 108
T = 1
t_sim = 200
NSim = 100
t_Sim = 200

p_sim = numpy.zeros(NSim)

for i in range(NSim):
    
    collisionData = pandas.read_csv("data_viriel/collisionData_LD_short{:d}.csv".format(i))
    [t,vDotr] = numpy.transpose(collisionData.as_matrix(['t','vDotr']))

    p_sim[i] = 1 + 1/(N*3*T*t_Sim) * numpy.sum(vDotr)

print (p_sim)

pyplot.figure(figsize=(8,5));
pyplot.plot(p_sim, '.');
pyplot.title("Petites Simulations");
pyplot.xlabel('simulation');
pyplot.ylabel('p');
pyplot.grid();
pyplot.show()

p_moy = numpy.sum(p_sim)/len(p_sim)
p_err = numpy.std(p_sim)/numpy.sqrt(len(p_sim))
print (p_moy)
print (p_err)

### Grandes simulations

In [ ]:
N = 108
T = 1
NSim = 10
t_Sim = 2000

p_sim = numpy.zeros(NSim)

for i in range(NSim):
    
    collisionData = pandas.read_csv("data_viriel/collisionData_LD_long{:d}.csv".format(i))
    [t,vDotr] = numpy.transpose(collisionData.as_matrix(['t','vDotr']))

    p_sim[i] = 1 + 1/(N*3*T*t_Sim) * numpy.sum(vDotr)

print (p_sim)

pyplot.figure(figsize=(8,5));
pyplot.plot(p_sim, '.');
pyplot.title("Grandes Simulations");
pyplot.xlabel('simulation');
pyplot.ylabel('p');
pyplot.grid();
pyplot.show()

p_moy = numpy.sum(p_sim)/len(p_sim)
p_err = numpy.std(p_sim)/numpy.sqrt(len(p_sim))
print (p_moy)
print (p_err)

On observe effectivement que concernant l'erreur statistique, il n'y a aucun avantage à utiliser une méthode plutôt que l'autre. 

## Réalisation d'un programme bouclant un grand nombre de simulations

Programme écrit dans le fichier "simulate.py"

Les pressions calculées par le programme sont enregistrées dans le fichier data_viriel/pressions.dat, qui est ensuite renommé en ajoutant un suffixe indiquant quelques paramètres utilisés.

Ci-dessous, les résultats pour un temps de simulation $t_{sim} = 1000$.

In [ ]:
f = open("data_viriel/pressures-tSim=1000.dat","r")

p = numpy.array([])

line = f.readline()
while line!="":
    p = numpy.append(p,float(line[:-1]))
    line = f.readline()
    
pyplot.figure(figsize=(8,5));
pyplot.plot(p, '.');
pyplot.title("Simulations pour $t_{sim} = 1000$");
pyplot.xlabel('simulation');
pyplot.ylabel('p');
pyplot.grid();
pyplot.show()
    

print ("Nombre de simulations =", len(p))
p_moy = numpy.sum(p)/len(p)
p_err = numpy.std(p)/numpy.sqrt(len(p))
print ("p_moy = {:.7f}".format(p_moy))
print ("p_err = {:.7f} ({:.1f}e-06)".format(p_err,p_err*1e6))

Avec autant de simulations, nous avons une erreur statistique assez petite pour pouvoir juger de la validité du résultat. Ce que nous remarquons, c'est que nous sommes loins de la valeur théorique $p = 1.003358$. 

Nous pouvons nous demander si la formule utilisée dans le script "simulate.py" pour calculer la pression est correcte.
En effet, dans la formule utilisée, ci-dessous, nous divisons par trois fois le nombre de particules $N$.

$$ p = 1+ \frac{1}{3NT t_{sim}} \sum_{t_{sim}} r_{ij}\cdot v_{ij} $$

Nous sommes en droit de nous demander si, comme dans la simulation la quantité de mouvement totale est conservée, nous ne devrions pas plutôt diviser par le nombre de degrés de libertés du système, dans ce cas-ci $3(N-1)$. C'est donc ce que nous essayons dans la cellule suivante.

In [ ]:
f = open("data_viriel/pressures-tSim=1000.dat","r")

p = numpy.array([])

line = f.readline()
while line!="":
    p = numpy.append(p,float(line[:-1]))
    line = f.readline()
    
p = 108/107*(p-1)+1 # corrige le calcul de la pression pour utiliser le nombre de degrés de liberté
    
pyplot.figure(figsize=(8,5));
pyplot.plot(p, '.');
pyplot.title("Simulations pour $t_{sim} = 1000$");
pyplot.xlabel('simulation');
pyplot.ylabel('p');
pyplot.grid();
pyplot.show()
    

print ("Nombre de simulations =", len(p))
p_moy = numpy.sum(p)/len(p)
p_err = numpy.std(p)/numpy.sqrt(len(p))
print ("p_moy = {:.7f}".format(p_moy))
print ("p_err = {:.7f} ({:.1f}e-06)".format(p_err,p_err*1e6))

On observe alors une amélioration du résultat, qui est déjà plus proche du résultat théorique. Cependant, ce n'est toujours pas la bonne valeur de $1.003358$. Une autre question que nous pouvons nous poser est si le temps de simulation est assez grand pour que le système atteigne l'équilibre. Nous essayons donc un temps de simulation plus grand et les résultats sont dans la cellule suivante.

In [ ]:
f = open("data_viriel/pressures-tSim=10000.dat","r")

p = numpy.array([])

line = f.readline()
while line!="":
    p = numpy.append(p,float(line[:-1]))
    line = f.readline()
    
p = 108/107*(p-1)+1
    
pyplot.figure(figsize=(8,5));
pyplot.plot(p, '.');
pyplot.title("Simulations pour $t_{sim} = 10000$");
pyplot.xlabel('simulation');
pyplot.ylabel('p');
pyplot.grid();
pyplot.show()
    

print ("Nombre de simulations =", len(p))
p_moy = numpy.sum(p)/len(p)
p_err = numpy.std(p)/numpy.sqrt(len(p))
print ("p_moy = {:.7f}".format(p_moy))
print ("p_err = {:.7f} ({:.1f}e-06)".format(p_err,p_err*1e6))

On observe bien cette fois-ci quelque chose de compatible avec le résultat théorique de $1.003358$. Il semble donc que l'équilibre est atteint sur une échelle de temps d'environ $1000$ et que la formule pour la pression doit utiliser le nombre de degrés de libertés du système.

## Relaxation du système vers l'équilibre

Nous allons ici tenter d'observer le système tendre vers l'équilibre au fur et à mesure que le temps progresse. Cela nous permettra de savoir à quel instant nous pouvons considérer que le système est à l'équilibre.

Pour ce faire, nous avons modifié le script "simulate.py" pour enregistrer la pression dans plusieurs intervalle de temps $[t,t+\Delta t[$. Nous simulerons sur un temps de $10000$ et calculerons la pression sur des intervalles de taille $100$. Le script modifié est "simulate-Deltas.py". Dans ce nouveau script, nous utilisons la bonne formule pour la pression qui utilise le nombre de degrés de libertés.

Les résultats, enregistrés dans le fichier "pressions-Deltas.dat", sont analysés dans la cellule suivante.

In [ ]:
f = open("data_viriel/pressures-Deltas.dat","r")

p = numpy.array([]) # liste des pressions sur les intervalles Delta_t pour toutes les simulations (matrice)

line = f.readline()
while line!="":
    
    idxPrevComma = -1
    p_sim = numpy.array([]) # pressions sur les intervalles Delta_t pour une simulation (vecteur)
    for i in range(len(line)-1):
        if line[i]==",":
            p_sim = numpy.append(p_sim,float(line[idxPrevComma+1:i]))
            idxPrevComma = i
    p_sim = numpy.append(p_sim,float(line[idxPrevComma+1:-1]))
    
    if len(p)==0: 
        p = [p_sim]
        #p = numpy.append(p,[p_sim])
    else:
        p = numpy.append(p,[p_sim], 0)
    line = f.readline()
    
p_moy = numpy.array([])
p_err = numpy.array([])

for i in range(len(p[0,:])): # parcourt les intervalles
    p_moy = numpy.append(p_moy,numpy.sum(p[:,i])/len(p[:,i]))
    p_err = numpy.append(p_err,numpy.std(p[:,i])/numpy.sqrt(len(p[:,i])))
    

#p_moy2 = numpy.array([])
#p_err2 = numpy.array([])
#num_sim = 417 # nombre de simulations à prendre en compte dans les stats
#shift_sim = 417 # prendre les simulations à partir de la shift_sim ème
#for i in range(len(p[0,:])): # parcourt les intervalles
#    p_moy2 = numpy.append(p_moy2,numpy.sum(p[shift_sim:shift_sim+num_sim,i])/len(p[shift_sim:shift_sim+num_sim,i]))
#    p_err2 = numpy.append(p_err2,numpy.std(p[shift_sim:shift_sim+num_sim,i])/numpy.sqrt(len(p[shift_sim:shift_sim+num_sim,i])))    

    
pyplot.figure(figsize=(8,5));
pyplot.errorbar(numpy.arange(len(p_moy)), p_moy, yerr=p_err, fmt='.');
#pyplot.errorbar(numpy.arange(len(p_moy2))*len(p_moy)/len(p_moy2), p_moy2, yerr=p_err2, fmt='.');
pyplot.title("Pressions moyennes au cours du temps");
pyplot.xlabel('$t/\Delta t$');
pyplot.ylabel('<p>');
pyplot.grid();
pyplot.show()

pyplot.figure(figsize=(8,5));
pyplot.plot(p_moy, '.');
pyplot.title("Pressions moyennes au cours du temps");
pyplot.xlabel('$t/\Delta t$');
pyplot.ylabel('<p>');
pyplot.grid();
pyplot.show()

pyplot.figure(figsize=(8,5));
pyplot.plot(p_err*1e6, '.');
pyplot.title("Erreurs sur les pressions moyennes au cours du temps");
pyplot.xlabel('$t/\Delta t$');
pyplot.ylabel('$\sigma\ x\ 10^{-6}$');
pyplot.grid();
pyplot.show()

print ("Nombre de simulations =", len(p[:,0]))

On remarque que le premier bin est à une pression nettement plus grande que les autres: pourquoi? 
Les autres points semblent suivrent une petite oscillation, mais celle-ci ne semble pas être la même suivant le nombre de simulation qu'on utilise pour les moyenne (!?), ces oscillations sont au-delà des barres d'erreur -> pas du hasard. Qu'est-ce qu'elles signifient ?